# First-order methods

Let $y_{j}$ denote approximation $y$ at grid points for $j = 1,2,3,\ldots,N$ and some positive integer values of $N$, we can use several methods. Consider the following IVP:
```{math}
:label: IVP_01
\frac{dy}{dt} = f(t,y),\quad a \leq t \leq b,~y(a) = y_{0},
```
In this section, we focus on several first-order methods to approximate the solution of this IVP numerically.

Here it is assumed that the grid points are equally distributed throughout the interval $[a, b]$. Let $N$ be a positive integer, we have, the following step size

\begin{align*}
h = \Delta t = \frac{b-a}{N}
\end{align*}
and each grid point can be identified as
\begin{align*}
t_{j} &= a +jh,& j = 0,1,2,\ldots,N.
\end{align*}

## (Forward) Euler method

Assume that $y\in C^{2}[a,b]$. Then, using Taylor's Theorem, for $j = 0, 1, 2, \ldots,N - 1$, we have,
\begin{align*}
y(t_{j+1}) = y(t_{j}) + hy'(t_{j}) + \frac{h^2}{2} y''(\xi_{j}),
\end{align*}
for some $\xi_{j} \in(t_{j},t_{j+1})$.Since $y(t)$ satisfies the differential equation {eq}`IVP_01`,
\begin{align*}
y(t_{j+1}) = y(t_{j}) + hf(t_{j}, y(t_{j})) + \frac{h^2}{2} y''(\xi_{j}).
\end{align*}
Therefore, Euler's method approximates the solution of the IVP {eq}`IVP_01` at each grid point starting from $x_0 = a$ where the solution is known as $y_0$. This process is carried forward iteratively until point $x_{n-1}$.
\begin{align*}
y_{j+1} = y(t_{j}) + hf(t_{j}, y_{j}).
\end{align*}

::::{tab-set}

:::{tab-item} Python Code
```python
import pandas as pd 
import numpy as np

def ForwardEuler(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output

    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    # loop
    for j in range(N):
        y[j+1] = y[j] + h*f(t[j], y[j])
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table
```
:::

:::{tab-item} MATLAB Code
```MATLAB
function [Table] = ForwardEuler(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
% loop
for j=1:N
    y(j+1) = y(j) + h*f(t(j), y(j));
end
Table = table(t,y);
end
```
:::

::::

<font color='Blue'><b>Example</b></font>: Consider the following IVP,
\begin{align*}
\begin{cases}
y'+2ty=te^{-t^2},\quad 0 \leq t \leq 1,\\
y(0) = 1,
\end{cases}
\end{align*}
with exact solution
\begin{align*}
y\left(t \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}.
\end{align*}
Use the forward Euler method for solving this IVP.

<font color='Green'><b>Solution</b></font>:

In [1]:
import sys
sys.path.insert(0,'..')
import hd_tools as hd

Loading BokehJS ...

In [2]:
import numpy as np
import pandas as pd
from hd_IVP_Algorithms import ForwardEuler  

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the exact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
y0 = 1
# Table
Table = ForwardEuler(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,1.000000,0.995000,4.9999e-03
2,0.200000,0.989900,0.980005,9.8953e-03
3,0.300000,0.969520,0.955058,1.4462e-02
4,0.400000,0.938767,0.920315,1.8452e-02
5,0.500000,0.897751,0.876151,2.1600e-02
6,0.600000,0.846916,0.823258,2.3658e-02
7,0.700000,0.787147,0.762720,2.4427e-02
8,0.800000,0.719830,0.696026,2.3804e-02
9,0.900000,0.646841,0.625026,2.1815e-02
10,1.000000,0.570447,0.551819,1.8627e-02


### Error Analysis


`````{admonition} Lemma
:class: tip

\begin{align*}
0 \leq (1 + x)^m \leq \exp(mx),\quad x \geq -1,~m>0.
\end{align*}
`````

`````{admonition} Lemma
:class: tip

Assume that $\{ a_{i} \}_{i =0}^{k}$ is a sequence satisfying $a_{0} \geq -t/s$ for some positive values $s$ and $t$, and
\begin{align*}
a_{i+1} \leq (1 + s)a_{i} + t,\quad i = 0,1,2,\ldots, k-1,
\end{align*}
then,
\begin{align*}
a_{i+1} \leq \exp((i+1)s) \left( a_{0} + \frac{t}{s} \right) - \frac{t}{s}.
\end{align*}

`````


```{admonition} Theorem

Assume that domain $D = \{ (t, y):~a\leq t \leq b,~\infty<y \infty\}$ and $f\in C(D)$. If $f$ satisfies a Lipschitz condition on $D$ in the variable $y$ with constant $L$ and assume that constant $M$ exists such that 
\begin{align*}
| y''(t)| \leq M, \quad t\in [a, b],
\end{align*}
where $y(t)$ denotes the unique solution to the IVP {eq}`IVP_01`.

Assume that $w_0$, $w_1$, ..., $w_N$ are the approximations generated by Euler's method for some positive integer $N.$ Then, for each
\begin{align*}
| y(t_{i}) - w_{i}| \leq \frac{hM}{2L}\left[ \exp(L(t_{i} - a)) -1 \right],\quad  i = 0, 1, 2, \ldots , N.
\end{align*}
`````

<font color='Blue'><b>Example</b></font>: In the previous example, investigate the order of convergence numerically.

<font color='Green'><b>Solution</b></font>:

For measuring error, we use the following measurement tool:
\begin{align*}
E_{h} = \max_{1 \leq j \leq \frac{b-a}{h}}\left| y(t_{j}) - y_{j}\right|
\end{align*}

In [3]:
h = [2**(-i) for i in range(3, 12)]
Cols = ['h', 'N', 'Eh']
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = ForwardEuler(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
    
display(Table.style.set_properties(subset=['h', 'N'],
                                   **{'background-color': 'PaleGreen', 'color': 'Black',
                                      'border-color': 'DarkGreen'}).format({'h':'{:.4e}','Eh':'{:.4e}'}))

,h,N,Eh
0,0.125000,8,3.0904e-02
1,0.062500,16,1.4889e-02
2,0.031250,32,7.2969e-03
3,0.015625,64,3.6122e-03
4,0.007812,128,1.7972e-03
5,0.003906,256,8.9636e-04
6,0.001953,512,4.4762e-04
7,0.000977,1024,2.2367e-04
8,0.000488,2048,1.1180e-04


In [4]:
from bokeh.plotting import show
hd.derivative_ConvergenceOrder(vecs = [Table['Eh'].values],
                            labels = ['Forward Euler method'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of convergence: %s' % 'Forward Euler method',
                            legend_orientation = 'horizontal', ylim = [0.9, 1.1])

## Backward (Implicit) Euler method

The backward Euler's method begins using the derivative at the next point $y(t_{i+1})$ instead of the current
point $y(t_{i})$:
\begin{align*}
y_{j+1} = y_{j} + hf(t_{j+1}, y_{j+1}).
\end{align*}
where $y_{j+1}$ for $j = 0, 1, 2, \ldots ,N - 1$ are approximated by solving the above equation in each step.

::::{tab-set}

:::{tab-item} Python Code
```python
import pandas as pd 
import numpy as np
from sympy import symbols, solve

def BackwardEuler(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output

    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    b = symbols('b')
    for j in range(N):
        y[j+1] = float(solve(y[j] + h*f(t[j+1], b) - b)[0])
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table
```
:::

:::{tab-item} MATLAB Code
```MATLAB
function [Table] = BackwardEuler(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
syms b
% loop
for j=1:N
    y(j+1) = vpasolve(y(j) + h*f(t(j+1), b) - b);
end
Table = table(t,y);
end
```
:::

::::

<font color='Blue'><b>Example</b></font>: Consider the following IVP,
\begin{align*}
\begin{cases}
y'+2ty=te^{-t^2},\quad 0 \leq t \leq 1,\\
y(0) = 1,
\end{cases}
\end{align*}
with exact solution
\begin{align*}
y\left(t \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}.
\end{align*}
Use the backward Euler method for solving this IVP. Also, investigate the order of convergence numerically.

<font color='Green'><b>Solution</b></font>:


In [5]:
from hd_IVP_Algorithms import BackwardEuler  

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1
# Table
Table = BackwardEuler(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,0.990099,0.995000,4.9016e-03
2,0.200000,0.970495,0.980005,9.5107e-03
3,0.300000,0.941427,0.955058,1.3631e-02
4,0.400000,0.903252,0.920315,1.7063e-02
5,0.500000,0.856539,0.876151,1.9612e-02
6,0.600000,0.802142,0.823258,2.1116e-02
7,0.700000,0.741251,0.762720,2.1469e-02
8,0.800000,0.675374,0.696026,2.0652e-02
9,0.900000,0.606281,0.625026,1.8745e-02
10,1.000000,0.535891,0.551819,1.5928e-02


Moreover, as for the error analysis, define,
\begin{align*}
E_{h} = \max_{1 \leq j \leq \frac{b-a}{h}}\left| y(t_{j}) - y_{j}\right|,
\end{align*}
then,

In [6]:
h = [2**(-i) for i in range(3, 8)]
Cols = ['h', 'N', 'Eh']
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = BackwardEuler(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
    
display(Table.style.set_properties(subset=['h', 'N'],
                                   **{'background-color': 'PaleGreen', 'color': 'Black',
                                      'border-color': 'DarkGreen'}).format({'h':'{:.4e}','Eh':'{:.4e}'}))

hd.derivative_ConvergenceOrder(vecs = [Table['Eh'].values],
                            labels = ['Backward Euler method'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of convergence: %s' % ' Backward Euler method',
                            legend_orientation = 'horizontal', ylim = [0.9, 1.1])

,h,N,Eh
0,0.125000,8,2.6255e-02
1,0.062500,16,1.3750e-02
2,0.031250,32,7.0121e-03
3,0.015625,64,3.5410e-03
4,0.007812,128,1.7793e-03


***
**References:**
1. Allaire, Grégoire, et al. Numerical linear algebra. Vol. 55. New York: Springer, 2008.
1. Burden, Richard L., and J. Douglas Faires. "Numerical analysis 8th ed." Thomson Brooks/Cole (2005).
1. Atkinson, Kendall E. An introduction to numerical analysis. John wiley & sons, 2008.
1. Khoury, Richard, and Douglas Wilhelm Harder. Numerical methods and modelling for engineering. Springer, 2016.
1. Zarowski, Christopher J. An introduction to numerical analysis for electrical and computer engineers. John Wiley & Sons, 2004.
1. [Euler method](https://en.wikipedia.org/wiki/Euler_method)
***